In [1]:
# imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tensorflow.keras import models, layers, optimizers, activations, callbacks

2024-05-05 18:26:45.920177: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-05 18:26:45.920423: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-05 18:26:45.923101: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-05 18:26:45.957627: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-05 18:26:46.539362: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
# read data
train = pd.read_parquet('playground-series-s4e5/train.parquet')
train.head()

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
0,5,8,5,8,6,4,4,3,3,4,...,5,3,3,5,4,7,5,7,3,0.445
1,6,7,4,4,8,8,3,5,4,6,...,7,2,0,3,5,3,3,4,3,0.450
2,6,5,6,7,3,7,1,5,4,5,...,7,3,7,5,6,8,2,3,3,0.530
3,3,4,6,5,4,8,4,7,6,8,...,2,4,7,4,4,6,5,7,5,0.535
4,5,3,2,6,4,4,3,3,3,3,...,2,2,6,6,4,1,2,3,5,0.415


In [3]:
# dependent and independent variable
X = train.drop(['FloodProbability'], axis=1)
y = train['FloodProbability']

In [4]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [5]:
model = models.Sequential(
    [layers.Input(shape=(20,)),
    layers.Dense(36, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(36, activation='relu'),
    layers.Dense(1, activation='relu')]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 36)             │           756 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │         4,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 36)             │         4,644 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            37 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,685 (104.24 KB)

 Trainable params: 26,685 (104.24 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
lronplateu = callbacks.ReduceLROnPlateau(monitor="val_loss",
    factor=0.5,
    patience=10,
    verbose=0,
    mode="auto",
    min_lr=0.00001
)

In [7]:
model.compile(optimizer=optimizers.Adam(learning_rate=0.0001), metrics=['r2_score'], loss='mean_absolute_error')

In [8]:
model.fit(X_train, y_train, validation_split=0.2, epochs=20, batch_size=32, callbacks=lronplateu)

Epoch 1/20
22360/22360 ━━━━━━━━━━━━━━━━━━━━ 20s 833us/step - loss: 0.0307 - r2_score: -0.7051 - val_loss: 0.0156 - val_r2_score: 0.8369 - learning_rate: 1.0000e-04
Epoch 2/20
22360/22360 ━━━━━━━━━━━━━━━━━━━━ 24s 1ms/step - loss: 0.0162 - r2_score: 0.8235 - val_loss: 0.0159 - val_r2_score: 0.8412 - learning_rate: 1.0000e-04
Epoch 3/20
22360/22360 ━━━━━━━━━━━━━━━━━━━━ 23s 1ms/step - loss: 0.0160 - r2_score: 0.8273 - val_loss: 0.0159 - val_r2_score: 0.8415 - learning_rate: 1.0000e-04
Epoch 4/20
22360/22360 ━━━━━━━━━━━━━━━━━━━━ 23s 1ms/step - loss: 0.0158 - r2_score: 0.8307 - val_loss: 0.0176 - val_r2_score: 0.7743 - learning_rate: 1.0000e-04
Epoch 5/20
22360/22360 ━━━━━━━━━━━━━━━━━━━━ 23s 1ms/step - loss: 0.0157 - r2_score: 0.8309 - val_loss: 0.0161 - val_r2_score: 0.8416 - learning_rate: 1.0000e-04
Epoch 6/20
22360/22360 ━━━━━━━━━━━━━━━━━━━━ 23s 1ms/step - loss: 0.0157 - r2_score: 0.8317 - val_loss: 0.0163 - val_r2_score: 0.8400 - learning_rate: 1.0000e-04
Epoch 7/20
22360/22360 ━━━━━━━━

In [9]:
model.evaluate(X_test, y_test)

6988/6988 ━━━━━━━━━━━━━━━━━━━━ 3s 448us/step - loss: 0.0155 - r2_score: 0.8393


[0.015504761599004269, 0.8386621475219727]

In [10]:
# test data
test = pd.read_csv('playground-series-s4e5/test.csv')
test.head()

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,IneffectiveDisasterPreparedness,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors
0,1117957,4,6,3,5,6,7,8,7,8,...,8,5,7,5,6,3,6,4,4,5
1,1117958,4,4,2,9,5,5,4,7,5,...,2,4,7,4,5,1,7,4,4,3
2,1117959,1,3,6,5,7,2,4,6,4,...,7,9,2,5,5,2,3,6,8,3
3,1117960,2,4,4,6,4,5,4,3,4,...,7,8,4,6,7,6,4,2,4,4
4,1117961,6,3,2,4,6,4,5,5,3,...,4,3,2,6,4,6,8,4,5,5


In [11]:
test_x = test.drop('id', axis=1)

In [12]:
test['FloodProbability'] = model.predict(test_x)

23291/23291 ━━━━━━━━━━━━━━━━━━━━ 9s 398us/step


In [13]:
submission_test1 = test[['id', 'FloodProbability']]

In [14]:
submission_test1

,id,FloodProbability
0,1117957,0.572984
1,1117958,0.449612
2,1117959,0.451126
3,1117960,0.460676
4,1117961,0.461991
...,...,...
745300,1863257,0.473404
745301,1863258,0.443685
745302,1863259,0.626982
745303,1863260,0.549253


In [17]:
submission_test1.to_csv('playground-series-s4e5/submission_test_ann_May5.csv', index=None)